In [49]:
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
# windows
# driver = webdriver.Chrome('F:\python\house_crawling\chromedriver.exe')

# MacOS
# driver = webdriver.Chrome('/Users/wanya/Simple_Car_model_crawling/chromedriver')

chrome_options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
  

driver.get('http://www.encar.com/db/db_carsinfo.do?method=newpricePop')
delay = 5
driver.implicitly_wait(delay)
"""리소스를 기다리는 시간(3초)"""



====== WebDriver manager ======
Current google-chrome version is 101.0.4951
Get LATEST chromedriver version for 101.0.4951 google-chrome
Driver [/Users/wanya/.wdm/drivers/chromedriver/mac64/101.0.4951.41/chromedriver] found in cache


'리소스를 기다리는 시간(3초)'

In [53]:
'''
1) 제조사   : //*[@id="mnfccd"]        | //*[@id="mnfccd"]
2) 모델     : //*[@id="mdlgroupcd"]  
3) 세부모델 : //*[@id="mdlcd"]         | //*[@id="mdlcd"]
4) 형식연도 : //*[@id="caryear"]       | //*[@id="caryear"]



//*[@id="mdlcd"]
//*[@id="caryear"]
//*[@id="mdlcd"]


'''
from selenium.webdriver.support.select import Select

# 1. 제조사
car_list = [[], [], [], []]

hide_keyword = ['제조사', '모델', '세부모델',' 형식연도']


select = Select(driver.find_element(By.ID, 'mnfccd'))
for opt in select.options:
    #get option text
    print("--------")
    print(opt.text)
    # 제조사 선택
    select.select_by_visible_text(opt.text) 
    
    # 2. 제조사-> 모델 선택
    try:
        select_2 = Select(driver.find_element(By.ID, 'mdlgroupcd'))
        for opt_2 in select_2.options:
            print(opt_2.text)
            select_2.select_by_visible_text(opt_2.text)
            
            try:
                # 3. 모델 -> 세부모델 선택
                select_3 = Select(driver.find_element(By.ID, 'mdlcd'))
                for opt_3 in select_3.options:
                    print(opt_3.text)
                    select_3.select_by_visible_text(opt_3.text)
                    select_4 = Select(driver.find_element(By.ID, 'caryear'))
                    for opt_4 in select_4.options:
                        # print(opt_4.text)

                        check_opt_2 = opt_4.text not in hide_keyword
                        if check_opt_2:
                            print(opt_4.text)
                            car_list.append([opt.text, opt_2.text, opt_3.text, opt_4.text])


            except Exception as E:
                # 3. 모델 -> 세부모델 선택 없을 때
                select_3 = Select(driver.find_element(By.ID, 'caryear'))
                for opt_3 in select_3.options:
                    # print(opt_3.text)
                    check_opt_3= opt_3.text not in hide_keyword
                    if check_opt_3:
                        print(opt_3.text)
                        car_list.append([opt.text, opt_2.text, '', opt_3.text])

                    
    except Exception as E:
        # 2-1. 제조사 -> 모델 x, 
        #       제조사 -> 세부모델 
        select_2 = Select(driver.find_element(By.ID, 'mdlcd'))
        for opt_2 in select_2.options:
            print(opt_2.text)
            select_2.select_by_visible_text(opt_2.text)
            
            # 3. 모델 -> 세부모델 선택
            select_3 = Select(driver.find_element(By.ID, 'caryear'))
            for opt_3 in select_3.options:
                # print(opt_3.text)

                                    # print(opt_3.text)
                check_opt_3= opt_3.text not in hide_keyword
                if check_opt_3:
                    print(opt_3.text)
                    car_list.append([opt.text, opt_2.text, '', opt_3.text])
                    # car_list.append([opt.text, opt_2.text, '', opt_3.text])

                # select_3.select_by_visible_text(opt_3.text)

          


        

# select by visible text

# select.select_by_visible_text('제네시스')

# select = Select(driver.find_element_by_id('mdlgroupcd'))

# # select by value 
# select.select_by_visible_text('G70')



--------
제조사
모델
세부모델
형식연도
--------
현대
모델
세부모델
형식연도
i30
세부모델
형식연도
2022
2021
2020
2019
2018
2017
2016
2015
2014
2013
2012
2011
2010
2009
2008
2007
2006
2005
2004
2003
2002
2001
2000
1999
1998
1997
1996
1995
1994
i30 (PD)
형식연도
2020
2019
2018
2017
2016
더 뉴 i30
형식연도
2016
2015
i30(신형)
형식연도
2015
2014
2013
2012
2011
i30 cw
형식연도
2011
2010
2009
2008
i30
형식연도
2011
2010
2009
2008
2007
i40
세부모델
형식연도
2022
2021
2020
2019
2018
2017
2016
2015
2014
2013
2012
2011
2010
2009
2008
2007
2006
2005
2004
2003
2002
2001
2000
1999
1998
1997
1996
1995
1994
더 뉴 i40
형식연도
2019
2018
2017
2016
2015
더 뉴 i40 살룬
형식연도
2019
2018
2017
2016
2015
i40 살룬
형식연도
2015
2014
2013
2012
i40
형식연도
2015
2014
2013
2012
2011
갤로퍼
세부모델
형식연도
2022
2021
2020
2019
2018
2017
2016
2015
2014
2013
2012
2011
2010
2009
2008
2007
2006
2005
2004
2003
2002
2001
2000
1999
1998
1997
1996
1995
1994
갤로퍼 2
형식연도
2003
2002
2001
2000
1999
1998
1997
갤로퍼
형식연도
1998
1997
1996
1995
1994
그랜저
세부모델
형식연도
2022
2021
2020
2019
2018
2017
2016
2015
2014
2013
2012
2011
2010
20

In [54]:
len(car_list)

9463

In [55]:
print(car_list)

[[], [], [], [], ['제조사', '모델', '세부모델', '형식연도'], ['현대', '모델', '세부모델', '형식연도'], ['현대', 'i30', '세부모델', '형식연도'], ['현대', 'i30', '세부모델', '2022'], ['현대', 'i30', '세부모델', '2021'], ['현대', 'i30', '세부모델', '2020'], ['현대', 'i30', '세부모델', '2019'], ['현대', 'i30', '세부모델', '2018'], ['현대', 'i30', '세부모델', '2017'], ['현대', 'i30', '세부모델', '2016'], ['현대', 'i30', '세부모델', '2015'], ['현대', 'i30', '세부모델', '2014'], ['현대', 'i30', '세부모델', '2013'], ['현대', 'i30', '세부모델', '2012'], ['현대', 'i30', '세부모델', '2011'], ['현대', 'i30', '세부모델', '2010'], ['현대', 'i30', '세부모델', '2009'], ['현대', 'i30', '세부모델', '2008'], ['현대', 'i30', '세부모델', '2007'], ['현대', 'i30', '세부모델', '2006'], ['현대', 'i30', '세부모델', '2005'], ['현대', 'i30', '세부모델', '2004'], ['현대', 'i30', '세부모델', '2003'], ['현대', 'i30', '세부모델', '2002'], ['현대', 'i30', '세부모델', '2001'], ['현대', 'i30', '세부모델', '2000'], ['현대', 'i30', '세부모델', '1999'], ['현대', 'i30', '세부모델', '1998'], ['현대', 'i30', '세부모델', '1997'], ['현대', 'i30', '세부모델', '1996'], ['현대', 'i30', '세부모델', '1995'], ['현대', 'i30', '세부모델', '

In [56]:
import pandas as pd
df = pd.DataFrame(car_list, columns=['maker', 'model', 'model_detail', 'year'])

In [57]:
df.head(1000)

,maker,model,model_detail,year
0,None,None,None,None
1,None,None,None,None
2,None,None,None,None
3,None,None,None,None
4,제조사,모델,세부모델,형식연도
...,...,...,...,...
995,현대,제네시스,제네시스,2012
996,현대,제네시스,제네시스,2011
997,현대,제네시스,제네시스,2010
998,현대,제네시스,제네시스,2009


In [59]:
df.isnull().sum()

maker           4
model           4
model_detail    4
year            4
dtype: int64

In [60]:
df_1 = df.dropna()

In [71]:
index = df_1[(df['maker'] == '제조사') | (df['model'] == '모델') | (df['model_detail'] == '세부모델') | (df['year'] == '형식연도')].index
index_model = df_1[df['model'] == '모델'].index
index_model_detail = df_1[df['model_detail'] == '세부모델'].index
index_year = df_1[df['year'] == '형식연도'].index


df_2 = df_1.drop(index)
df_2

/var/folders/f8/11bx78sd5gz2q2ppw4079pjc0000gn/T/ipykernel_63993/4113140939.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  index = df_1[(df['maker'] == '제조사') | (df['model'] == '모델') | (df['model_detail'] == '세부모델') | (df['year'] == '형식연도')].index
/var/folders/f8/11bx78sd5gz2q2ppw4079pjc0000gn/T/ipykernel_63993/4113140939.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  index_model = df_1[df['model'] == '모델'].index
/var/folders/f8/11bx78sd5gz2q2ppw4079pjc0000gn/T/ipykernel_63993/4113140939.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  index_model_detail = df_1[df['model_detail'] == '세부모델'].index
/var/folders/f8/11bx78sd5gz2q2ppw4079pjc0000gn/T/ipykernel_63993/4113140939.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  index_year = df_1[df['year'] == '형식연도'].index


,maker,model,model_detail,year
37,현대,i30,i30 (PD),2020
38,현대,i30,i30 (PD),2019
39,현대,i30,i30 (PD),2018
40,현대,i30,i30 (PD),2017
41,현대,i30,i30 (PD),2016
...,...,...,...,...
9458,혼다,시빅,,2010
9459,혼다,시빅,,2009
9460,혼다,시빅,,2008
9461,혼다,시빅,,2007


In [73]:
df_2.to_csv('encar_list.csv', sep=',', na_rep='') 

